In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from typing import List, Union



# 1. Defina as features (variáveis de entrada) na ordem correta


In [ ]:
# Define as features, na ordem correta
FEATURE_NAMES = [
    'COD_UF_EMIT', 'TIP_FIN_NFE', 'CEST_COMPLETO', 'COD_CST', 'NCM_COMPLETO', 'CFOP_COMPLETO', 'EMIT_CNAE_COMPLETO', 'EMIT_CRT', 
    'EMIT_IND_SN', 'DEST_CNAE_COMPLETO', 'DEST_SITUACAO', 'DEST_IND_SN', 'DEST_POSSUI_IE', 'EMIT_CNAE_DIVISAO', 'EMIT_CNAE_GRUPO', 
    'EMIT_CNAE_CLASSE', 'DEST_CNAE_DIVISAO', 'DEST_CNAE_GRUPO', 'DEST_CNAE_CLASSE', 'NCM_CAPITULO', 'NCM_POSICAO', 'NCM_SUBPOSICAO', 
    'CFOP_NATUREZA', 'CFOP_OPERACAO', 'POSSUI_CEST', 'CEST_SEGMENTO', 'CEST_ITEM']

# Features que devem ser tratadas como Categóricas (para corrigir o erro)
CATEGORICAL_FEATURES = [
    'COD_UF_EMIT', 'TIP_FIN_NFE', 'CEST_COMPLETO', 'COD_CST', 'NCM_COMPLETO', 'CFOP_COMPLETO', 'EMIT_CNAE_COMPLETO', 'EMIT_CRT', 
    'EMIT_IND_SN', 'DEST_CNAE_COMPLETO', 'DEST_SITUACAO', 'DEST_IND_SN', 'DEST_POSSUI_IE', 'EMIT_CNAE_DIVISAO', 'EMIT_CNAE_GRUPO', 
    'EMIT_CNAE_CLASSE', 'DEST_CNAE_DIVISAO', 'DEST_CNAE_GRUPO', 'DEST_CNAE_CLASSE', 'NCM_CAPITULO', 'NCM_POSICAO', 'NCM_SUBPOSICAO', 
    'CFOP_NATUREZA', 'CFOP_OPERACAO', 'POSSUI_CEST', 'CEST_SEGMENTO', 'CEST_ITEM'
]

# 2. Caminho do arquivo do modelo


In [ ]:
MODEL_FILE_PATH = "modelo_lgbm 3.txt"


# 3. Função principal para carregar o modelo e fazer a predição


In [ ]:
def carregar_e_classificar(dados_x: pd.DataFrame) -> Union[np.ndarray, int]:
    """
    Carrega o modelo LightGBM, realiza o pré-processamento necessário
    e faz a classificação dos dados fornecidos.
    """
    try:
        # Carrega o modelo (Booster)
        bst = lgb.Booster(model_file=MODEL_FILE_PATH)
    except lgb.basic.LightGBMError as e:
        print(f"Erro ao carregar o modelo: {e}")
        return None
    except Exception as e:
        print(f"Ocorreu um erro inesperado: {e}")
        return None

    # Garante que as colunas do DataFrame estão na ordem correta
    dados_para_predicao = dados_x[FEATURE_NAMES].copy()

    # **ETAPA DE CORREÇÃO:** Converte as colunas categóricas para o tipo 'category'
    for col in CATEGORICAL_FEATURES:
        if col in dados_para_predicao.columns:
            # Converte para string antes de category para tratar valores mistos (num/str)
            dados_para_predicao[col] = dados_para_predicao[col].astype(str).astype('category')
    
    # Realiza a predição. O LightGBM agora deve reconhecer as colunas categóricas.
    probabilidades = bst.predict(dados_para_predicao)

    # A classificação final é a classe com a maior probabilidade.
    classificacao_y = np.argmax(probabilidades, axis=1)

    # Retorna o valor ou a lista de valores
    if len(classificacao_y) == 1:
        return classificacao_y[0]
    return classificacao_y

# 4. Exemplo de Uso


In [ ]:
import pandas as pd
from io import StringIO

# A string de dados fornecida, formatada para simular a entrada de um arquivo tabulado
dados_string = """
SEQ_NFE\tCOD_UF_EMIT\tTIP_FIN_NFE\tID_PROD\tCEST_COMPLETO\tCOD_CST\tNCM_COMPLETO\tCFOP_COMPLETO\tEMIT_CNAE_COMPLETO\tEMIT_CRT\tEMIT_IND_SN\tDEST_CNAE_COMPLETO\tDEST_SITUACAO\tTIPO_IMPOSTO\tDEST_IND_SN\tDEST_POSSUI_IE\tEMIT_CNAE_DIVISAO\tEMIT_CNAE_GRUPO\tEMIT_CNAE_CLASSE\tDEST_CNAE_DIVISAO\tDEST_CNAE_GRUPO\tDEST_CNAE_CLASSE\tNCM_CAPITULO\tNCM_POSICAO\tNCM_SUBPOSICAO\tCFOP_NATUREZA\tCFOP_OPERACAO\tPOSSUI_CEST\tCEST_SEGMENTO\tCEST_ITEM
435959728\tSP\t1\t2928998889\t0\t0\t96082000\t6102\tAUSENTE\t1\tS\t55108010\t2\tSTDIF\t0\t1\tAUSENTE\tAUSENTE\tAUSENTE\t55\t551\t5510\t96\t9608\t960820\t6\t102\t0\tAUSENTE\tAUSENTE
435959728\tSP\t1\t2928998890\t0\t0\t44191900\t6101\tAUSENTE\t1\tS\t55108010\t2\tANT\t0\t1\tAUSENTE\tAUSENTE\tAUSENTE\t55\t551\t5510\t44\t4419\t441919\t6\t101\t0\tAUSENTE\tAUSENTE
"""

# Usando StringIO para tratar a string como um arquivo
# O parâmetro 'sep='\t'' indica que a tabulação é o separador
df = pd.read_csv(StringIO(dados_string), sep='\t')

print("DataFrame 'df' criado com sucesso:")
print("-" * 50)
print(df.head())
print("-" * 50)
print(f"Dimensão do DataFrame: {df.shape}")
print(f"Colunas (Total {len(df.columns)}): {df.columns.tolist()}")


# Realiza a classificação
resultado_y = carregar_e_classificar(df)

if resultado_y is not None:
    print("--- Resultado da Classificação (Y) ---")
    if isinstance(resultado_y, int):
            print(f"Classificação prevista: {resultado_y}")
    else:
        df['CLASSIFICACAO_Y'] = resultado_y
        print(df[['CLASSIFICACAO_Y']])
        print(f"\nResultado completo (array): {resultado_y}")